# Introduction to text classification

We will illustrate basic text classification approchaches on [
SMS Spam Collection](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection) data set. 
The SMS Spam Collection is a set of SMS tagged messages that have been collected for SMS Spam research. It contains one set of SMS messages in English of 5,574 messages, tagged acording being ham (legitimate) or spam.

In [1]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/mlcollege/natural-language-processing/master/data/spam.csv', sep='\t')
df.head()

,Target,Text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## Preparation of train and test data sets

Separate and rename target values.

In [0]:
data = df['Text']
target = df['Target'].replace('ham', 1).replace('spam', 0)
names = ['spam', 'ham']
print(data[:5])
print(target[:5])

Shuffle the data and split it to train and test parts.

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2)
print('Train size: {}'.format(len(X_train)))
print('Test size: {}'.format(len(X_test)))

## Data preprocessing

Tokenize the texts. Experiment with various tokenizers from the [NLTK](http://www.nltk.org/api/nltk.tokenize.html) library.

In [0]:
from nltk.tokenize.casual import casual_tokenize

sms = data[4]
print(sms)

tokenizer = lambda text: casual_tokenize(text, preserve_case=False)

print(tokenizer(sms))

In [0]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
print(stopwords.words('english'))
stopword_tokenizer = lambda text: [w for w in tokenizer(text) if not w in set(stopwords.words('english'))]

In [0]:
stopword_tokenizer(sms)

Convert tokens to their stems. Experiment with stemmers and lemmatizers from the [NLTK](http://www.nltk.org/api/nltk.stem.html) library.

In [0]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

stem_tokenizer = lambda text: [stemmer.stem(w) for w in stopword_tokenizer(text)]

print (stem_tokenizer(sms))

Fit a vectorizer which converts texts to count vectors.

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(tokenizer=stem_tokenizer)
vectorizer.fit(X_train)
print (vectorizer.transform([sms]))

Convert count vectors to TFIDF

In [0]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()
tfidf_transformer.fit(vectorizer.transform(X_train))
print(tfidf_transformer.transform(vectorizer.transform([sms])))

## Classification

Train a classifier using the following models:
* [Logistic regression](http://scikit-learn.org/0.15/modules/generated/sklearn.linear_model.LogisticRegression.html)
* [Gradient Boosted Trees](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html) (Experiment with different depths and number of trees)
* [Support Vector Machines](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC) (experiment with different kernels)

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier

clf_pipeline = Pipeline([('vec', vectorizer),
                         ('tfidf', tfidf_transformer),
                         #('lr', LogisticRegression())
                         #('gbc', GradientBoostingClassifier(n_estimators=100, max_depth=4))
                         ('svm', svm.SVC(kernel='linear'))
                        ])
clf_pipeline.fit(X_train, y_train)

## Evaluation

Compute common classification metrics and evaluate the models. Decide which model performs best on the given problem.

In [0]:
from sklearn import metrics
from sklearn.metrics import accuracy_score

y_pred = clf_pipeline.predict(X_test)

print ("Test accuracy: {:.5f}".format(accuracy_score(y_test, y_pred)))
print ()
print(metrics.classification_report(y_test, y_pred, digits=4))

In [0]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, y_pred))

In [0]:
y_pred = clf_pipeline.predict(X_train)

print ("Train accuracy: {:.2f}".format(accuracy_score(y_train, y_pred)))
print ()
print(metrics.classification_report(y_train, y_pred, digits=4))